In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
os.getcwd()

'C:\\Users\\Jalal\\Box\\GNN Project\\Code files'

In [4]:
#pip install netaddr

In [5]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [6]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Wednesday_Packet_Level_Data.pkl')

In [7]:
df1 = wed_data.copy()

In [8]:
del wed_data

In [9]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [10]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499258820.014743000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000034a7b6400037062efd173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801003ab15920000010108...,00,fwd,1.0,1.0
1,1499258820.039091000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000722a7b740003706280e173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801803ab74250000010108...,485454502f312e3120323030204f4b0d0a536572766572...,bwd,1.0,1.0
2,1499258820.039116000,192.168.10.17,23.60.139.27,47757,80,12035,435,00c1b114eb310023ae9b95670800,45000034fcd740004006d0dbc0a80a11173c8b1b,ba8d00505d3f78e3a5299c698010010011280000010108...,00,fwd,1.0,1.0
3,1499258820.082489000,192.168.10.12,208.185.50.90,51774,443,11808,4580,00c1b114eb31b8ac6f3604e30800,4500002804414000400668c7c0a80a0cd0b9325a,ca3e01bb4c69a4a67cf3967050109eb472ea0000,000000000000,fwd,1.0,1.0
4,1499258820.084153000,192.168.10.17,23.208.97.23,37115,443,12034,349,00c1b114eb310023ae9b95670800,4500017a021640004006f3c7c0a80a1117d06117,90fb01bb96268b0ed08217e7801801356d9e0000010108...,1703030141000000000000000174b7a44755da49b50c35...,fwd,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334902,1499276130.701758000,172.16.0.1,192.168.10.51,443,53199,243533,0,b8ac6f360ba800c1b114eb310800,4500003cc45240003f06007dac100001c0a80a33,01bbcfcf167451881e1ff3d6a012fe884e0a0000020405...,00,bwd,0.0,0.0
11334903,1499276130.702180000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,45000034034d40003f06c18aac100001c0a80a33,01bbcfcf167451891e1ff5db8010000879580000010108...,00,bwd,0.0,0.0
11334904,1499276130.702964000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,450000bd034e40003f06c100ac100001c0a80a33,01bbcfcf167451891e1ff5db801800084ed60000010108...,16030300510200004d0303587c9e338fe3870b8fd3c17c...,bwd,0.0,0.0
11334905,1499276130.710822000,172.16.0.1,192.168.10.51,443,53199,243533,138,b8ac6f360ba800c1b114eb310800,45000034034f40003f06c188ac100001c0a80a33,01bbcfcf167452121e1ffa1f80100008748a0000010108...,00,bwd,0.0,0.0


In [11]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [12]:
df1['Class Label'].value_counts()

0.0    8845368
3.0    2246818
4.0     106177
5.0      49296
1.0      47586
2.0      39662
Name: Class Label, dtype: int64

In [13]:
df = df1[df1['Class Label']== 3]

In [14]:
df = df.reset_index(inplace = False)

In [15]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,87248,1499259644.093729000,192.168.10.17,173.223.152.83,53650,443,17507,22194,00c1b114eb310023ae9b95670800,45000034f773400040063264c0a80a11addf9853,d19201bb868fb1984046bd898010059f85290000010108...,00,fwd,1.0,3.0
1,87249,1499259644.093731000,173.223.152.83,192.168.10.17,443,53654,17511,434653,0023ae9b956700c1b114eb310800,4500071519eb40003506140caddf9853c0a80a11,01bbd19641f452fa2e33048280180aff17f40000010108...,17030306dcde50c8893ac7380387f0630725076944cced...,fwd,1.0,3.0
2,87250,1499259644.094004000,192.168.10.17,173.223.152.83,53650,443,17507,22194,00c1b114eb310023ae9b95670800,450001b2f7744000400630e5c0a80a11addf9853,d19201bb868fb1984046bd89801805a45c740000010108...,1703030179000000000000003a47ab428cb6c48f9e63e8...,bwd,1.0,3.0
3,87251,1499259644.094055000,173.223.152.83,192.168.10.17,443,53654,17511,436414,0023ae9b956700c1b114eb310800,450005dc19ed400035061543addf9853c0a80a11,01bbd19641f459db2e33048280100aff4b5d0000010108...,1703031d24de50c8893ac73804b8d04c7dedaecca2534f...,bwd,1.0,3.0
4,87252,1499259644.094096000,192.168.10.17,173.223.152.83,53654,443,17511,21881,00c1b114eb310023ae9b95670800,4500003404344000400625a4c0a80a11addf9853,d19601bb2e33048241f45f8380100598e6f70000010108...,00,fwd,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246813,2334061,1499262905.888524000,192.168.10.50,172.16.0.1,80,41380,145490,0,00c1b114eb310019b90a69f10800,4500003c000040004006c3d0c0a80a32ac100001,0050a1a4cb8b1e87d7cb0d8ca012712064960000020405...,00,fwd,1.0,3.0
2246814,2334062,1499262905.888549000,192.168.10.50,172.16.0.1,80,41348,145474,1,00c1b114eb310019b90a69f10800,450000342f71400040069467c0a80a32ac100001,0050a18400db22bf0753237f801000eb84b70000010108...,00,fwd,1.0,3.0
2246815,2334063,1499262905.888550000,192.168.10.50,172.16.0.1,80,41352,145473,1,00c1b114eb310019b90a69f10800,45000034086940004006bb6fc0a80a32ac100001,0050a1883e1e05a7fa56683d801000eb2cc60000010108...,00,fwd,1.0,3.0
2246816,2334064,1499262905.888551000,151.101.44.65,192.168.10.15,443,54301,135532,7427431,001e4fd4ca2800c1b114eb310800,45000b900e85400035069d8597652c41c0a80a0f,01bbd41d128c11d983b9ff4b5010010999e00000,187620657d2b0cfeea066d1e195bd7ee523fe0f288eb8d...,fwd,1.0,3.0


In [16]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [17]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [18]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [19]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [20]:
# 70% of benign data for train, 30% for test benign

In [21]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [22]:
#len(imlist)

In [23]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [24]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [25]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [26]:
#len(streams_lst1)

In [27]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.17,173.223.152.83,53650,443,1499259644.093729000,17507,22194,fwd,"[0.0, 0.20392156862745098, 0.9686274509803922,..."
1,173.223.152.83,192.168.10.17,443,53654,1499259644.093731000,17511,434653,fwd,"[0.027450980392156862, 0.08235294117647059, 0...."
2,192.168.10.17,173.223.152.83,53650,443,1499259644.094004000,17507,22194,bwd,"[0.00392156862745098, 0.6980392156862745, 0.96..."
3,173.223.152.83,192.168.10.17,443,53654,1499259644.094055000,17511,436414,bwd,"[0.0196078431372549, 0.8627450980392157, 0.098..."
4,192.168.10.17,173.223.152.83,53654,443,1499259644.094096000,17511,21881,fwd,"[0.0, 0.20392156862745098, 0.01568627450980392..."
...,...,...,...,...,...,...,...,...,...
2246813,192.168.10.50,172.16.0.1,80,41380,1499262905.888524000,145490,0,fwd,"[0.0, 0.23529411764705882, 0.0, 0.0, 0.2509803..."
2246814,192.168.10.50,172.16.0.1,80,41348,1499262905.888549000,145474,1,fwd,"[0.0, 0.20392156862745098, 0.1843137254901961,..."
2246815,192.168.10.50,172.16.0.1,80,41352,1499262905.888550000,145473,1,fwd,"[0.0, 0.20392156862745098, 0.03137254901960784..."
2246816,151.101.44.65,192.168.10.15,443,54301,1499262905.888551000,135532,7427431,fwd,"[0.043137254901960784, 0.5647058823529412, 0.0..."


In [28]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Wednesday_hulk_all/'

for PKT in range(2, 16):
    
    file_name = f'wednesday_hulk_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 128566
